In [30]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [31]:
data = pd.read_csv(r"C:\Users\Kirthik\Downloads\Bank_Personal_Loan_Modelling.csv")
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [32]:
train_data = data.sample(frac=0.8, random_state=1)
test_data = data.drop(train_data.index)


In [33]:
train_features = train_data.drop(columns=['ID', 'ZIP Code', 'Personal Loan']).to_numpy()
train_labels = train_data['Personal Loan'].to_numpy().reshape(-1, 1)
test_features = test_data.drop(columns=['ID', 'ZIP Code', 'Personal Loan']).to_numpy()
test_labels = test_data['Personal Loan'].to_numpy().reshape(-1, 1)


In [37]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.hidden_weights = np.random.randn(self.input_size, self.hidden_size)
        self.hidden_biases = np.zeros((1, self.hidden_size))
        self.output_weights = np.random.randn(self.hidden_size, self.output_size)
        self.output_biases = np.zeros((1, self.output_size))
        self.hidden_layer_activation = None

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward(self, inputs):
        hidden_layer = np.dot(inputs, self.hidden_weights) + self.hidden_biases
        self.hidden_layer_activation = self.sigmoid(hidden_layer)
        output_layer = np.dot(self.hidden_layer_activation, self.output_weights) + self.output_biases
        output_layer_activation = self.sigmoid(output_layer)
        return output_layer_activation

    def backward(self, inputs, labels, outputs):
        output_errors = labels - outputs
        output_gradients = output_errors * outputs * (1 - outputs)
        hidden_errors = np.dot(output_gradients, self.output_weights.T)
        hidden_gradients = hidden_errors * self.hidden_layer_activation * (1 - self.hidden_layer_activation)
        self.output_weights += np.dot(self.hidden_layer_activation.T, output_gradients)
        self.output_biases += np.sum(output_gradients, axis=0, keepdims=True)
        self.hidden_weights += np.dot(inputs.T, hidden_gradients)
        self.hidden_biases += np.sum(hidden_gradients, axis=0, keepdims=True)

    def train(self, inputs, labels, num_epochs, learning_rate):
        for epoch in range(num_epochs):
            outputs = self.forward(inputs)
            self.backward(inputs, labels, outputs)
            if epoch % 100 == 0:
                loss = np.mean(np.square(labels - outputs))
                print(f"Epoch {epoch}, Loss: {loss}")
        return self

    def predict(self, inputs):
        outputs = self.forward(inputs)
        return (outputs > 0.5).astype(int)

# Train the neural network
input_size = train_features.shape[1]
hidden_size = 10
output_size = 1
num_epochs = 1000
learning_rate = 0.1

nn = NeuralNetwork(input_size, hidden_size, output_size)
nn.train(train_features, train_labels, num_epochs, learning_rate)

# Compute the accuracy on the test set
test_predictions = nn.predict(test_features)
accuracy = np.mean(test_predictions == test_labels)
print(f"Test Accuracy: {accuracy}")

Epoch 0, Loss: 0.2421156914668417


C:\Users\Kirthik\AppData\Local\Temp\ipykernel_3304\1404430554.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 100, Loss: 0.0945
Epoch 200, Loss: 0.0945
Epoch 300, Loss: 0.0945
Epoch 400, Loss: 0.0945
Epoch 500, Loss: 0.0945
Epoch 600, Loss: 0.0945
Epoch 700, Loss: 0.0945
Epoch 800, Loss: 0.0945
Epoch 900, Loss: 0.0945
Test Accuracy: 0.898
